## 2a. Evidence - Fairnesss QAS Measurements

Evidence collected in this section checks for the Fairness scenario defined in the previous step. Note that some functions will be loaded from external Python files.

### Initialize MLTE Context

MLTE contains a global context that manages the currently active _session_. Initializing the context tells MLTE how to store all of the artifacts that it produces. This import will also set up global constants related to folders and model to use.

In [1]:
# Sets up context for the model being used, sets up constants related to folders and model data to be used.
from demo.scenarios.session import *

### Helper Functions
General functions and external imports.

In [6]:
# General functions.

from demo.scenarios import garden
import numpy as np


def load_data(data_folder: str):
    """Loads all garden data results and taxonomy categories."""
    df_results = garden.load_base_results(data_folder,"predictions_test.csv")
    df_results.head()

    # Load the taxonomic data and merge with results.
    df_info = garden.load_taxonomy(data_folder)
    df_results.rename(columns={"label": "Label"}, inplace=True)
    df_all = garden.merge_taxonomy_with_results(df_results, df_info)

    return df_info, df_all


def split_data(df_info, df_all,population_size=100):
    """Splits the data into 3 different populations to evaluate them."""
    df_gardenpop = df_info.copy()
    df_gardenpop["Population1"] = (
        np.around(
            np.random.dirichlet(np.ones(df_gardenpop.shape[0]), size=1)[0],
            decimals=3,
        )
        * population_size
    ).astype(int)
    df_gardenpop["Population2"] = (
        np.around(
            np.random.dirichlet(np.ones(df_gardenpop.shape[0]), size=1)[0],
            decimals=3,
        )
        * population_size
    ).astype(int)
    df_gardenpop["Population3"] = (
        np.around(
            np.random.dirichlet(np.ones(df_gardenpop.shape[0]), size=1)[0],
            decimals=3,
        )
        * population_size
    ).astype(int)
    #df_gardenpop
    print("Hello")#df_gardenpop["Population2"])


    print(df_gardenpop["Population2"])
    # build populations from test data set that match the garden compositions
    from random import choices

    # build 3 gardens with populations of population_size.
    pop_names = ["Population1", "Population2", "Population3"]
    gardenpops = np.zeros((3, population_size), int)
    gardenmems = np.zeros((3, population_size), int)

    for j in range(population_size):
        for i in range(len(df_gardenpop)):
            my_flower = df_gardenpop.iloc[i]["Common Name"]

            for g in range(3):
                n_choices = df_gardenpop.iloc[i][pop_names[g]]
                my_choices = df_all[df_all["Common Name"] == my_flower][
                    "model correct"
                ].to_list()
                #print(f"{n_choices} {my_choices}")
                my_selection = choices(my_choices, k=n_choices)

                gardenpops[g][j] += sum(my_selection)
                gardenmems[g][j] += len(my_selection)

    gardenpops

    return gardenpops, gardenmems


def calculate_model_performance_acc(gardenpops, gardenmems,population_size=100):
    """Get accucray of models across the garden populations"""
    gardenacc = np.zeros((3, population_size), float)
    for i in range(population_size):
        for g in range(3):
            gardenacc[g][i] = gardenpops[g][i] / gardenmems[g][i]
    gardenacc

    model_performance_acc = []
    for g in range(3):
        avg = round(np.average(gardenacc[g][:]), 3)
        std = round(np.std(gardenacc[g][:]), 3)
        min = round(np.amin(gardenacc[g][:]), 3)
        max = round(np.amax(gardenacc[g][:]), 3)
        model_performance_acc.append(round(avg, 3))

        print("%1d %1.3f %1.3f %1.3f %1.3f" % (g, avg, std, min, max))

    return model_performance_acc

In [7]:
# Prepare the data. For this section, instead of executing the model, we will use CSV files containing the results of an already executed run of the model.
data = load_data(DATASETS_DIR)
print("Splitting Data")
#print(data)
split_data = split_data(data[0], data[1])

     Unnamed: 0  Group  Count              Common Name  Other Name Risk  \
0             0     73     40                 primrose         NaN  NaN   
1             1     22     40         canterbury_bells         NaN  NaN   
2             2     63     40              moon_orchid         NaN  NaN   
3             3     37     40                fire_lily         NaN  NaN   
4             4     75     40  prince_of_wales_feather         NaN   G4   
..          ...    ...    ...                      ...         ...  ...   
97           97     98    184               watercress  nasturtium  NaN   
98           98     97    194               water_lily         NaN  NaN   
99           99     96    196               wallflower    erysimum  NaN   
100         100     67    251           passion_flower         NaN  NaN   
101         101     70    258                  petunia         NaN  NaN   

     Kingdom         Phylum       Clade1    Clade2    Clade3           Order  \
0    Plantae  Trach

In [8]:
data

(     Unnamed: 0  Group  Count       Common Name  Other Name  Risk  Kingdom  \
 0            14      0     43  alpine_sea_holly         NaN    NT  Plantae   
 1            80      1    105         anthurium         NaN  None  Plantae   
 2            62      2     78         artichoke         NaN  None  Plantae   
 3            35      3     56         arum_lily         NaN  None  Plantae   
 4            78      4     96            azalea         NaN  None  Plantae   
 ..          ...    ...    ...               ...         ...   ...      ...   
 97           98     97    194        water_lily         NaN  None  Plantae   
 98           97     98    184        watercress  nasturtium  None  Plantae   
 99           70     99     85        wild_pansy         NaN  None  Plantae   
 100          57    100     68        windflower     anemone  None  Plantae   
 101          24    101     49       yellow_iris         NaN  None  Plantae   
 
             Phylum       Clade1    Clade2    Clad

### Measurements

In this first example, we simply wrap the output from `accuracy_score` with a custom `Result` type to cope with the output of a third-party library that is not supported by a MLTE builtin.

In [9]:
from demo.scenarios.values.multiple_accuracy import MultipleAccuracy
from mlte.measurement.external_measurement import ExternalMeasurement

# Evaluate accuracy, identifier has to be the same one defined in the Spec.
accuracy_measurement = ExternalMeasurement(
    "accuracy across gardens", MultipleAccuracy, calculate_model_performance_acc
)
accuracy = accuracy_measurement.evaluate(split_data[0], split_data[1])

# Inspect value
print(accuracy)

# Save to artifact store
accuracy.save(force=True)

0 0.951 0.023 0.900 1.000
1 0.958 0.023 0.885 1.000
2 0.950 0.023 0.906 1.000
[np.float64(0.951), np.float64(0.958), np.float64(0.95)]
